In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import pruner as P
import scoring as S

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
device = 'cuda:0'

model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Linear(2, 1)
)

model = model.to(device)

# pr have the masks of the model and manage the spasity
# selectable scopes: 'layer', 'neuron', 'global'
pr = P.Pruner(model, 'global', weight=True, bias=False, norm=False)

optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_func = nn.MSELoss()


In [9]:
scores = S.magnitude(model, pr)

# update_mask updates masks and parameters using the scores
pr.update_mask(model, scores, sparsity=0.5)

for name, param in model.named_parameters():
    if 'weight' in name:
        print(param.data)


compressing to 12.5% ... done.
tensor([[0., 0.],
        [0., 0.]], device='cuda:0')
tensor([[0.0000, 0.6096]], device='cuda:0')


In [4]:
model.train()

x = torch.rand(1, 2).to(device)
y = torch.Tensor([[0.5, 0.5]]).to(device)

outputs = model(x)
loss = loss_func(outputs, y)

model.zero_grad()
loss.backward()

# freeze the parameters before optimizer.step()
pr.freeze_grad(model)
        
optimizer.step()



/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [5]:
for name, param in model.named_parameters():
    if 'weight' in name:
        print(param.data)

tensor([[0.5060, 0.0000],
        [0.0000, 0.2111]], device='cuda:0')
tensor([[0.0000, 0.6704]], device='cuda:0')
